## 西太平洋副热带高压指数的预测模块
---
说明: 读入 500 hPa 位势高度和 u 方向风场数据文件, 计算给定年月的副热带高压指数. 
(TODO)如果给定的年月在数据中还没有发生, 则启动预测模块.

- 面积指数: 在 10N 以北 110E - 180 范围内, 500 hPa 位势高度场上所有位势高度不小于 588 dagpm 的格点围成的面积总和.
- 强度指数: 在 10N 以北 110E - 180 范围内, 500 hPa 位势高度场上所有位势高度不小于 588 dagpm 的格点围成的面积与该格点高度值减去 587 dagpm 差值的乘积的总和.
- 脊线指数: 在 10N 以北 110 - 150E 范围内, 588 dagpm 等值线所包围的副热带高压体内纬向风 u = 0 且 du/dy > 0 的特征线所在纬度位置的平均值; 若不存在 588 dagpm 等值线, 则定义 584 dagpm 等值线范围内纬向风 u = 0, 且 du/dy > 0 的特征线所在纬度位置的平均值; 若在某月不存在 584 dagpm 等值线, 则以该月的 1951 - 2010 年历史最小值代替.
- 西伸脊点: 在 90E - 180 范围内, 588 dagpm 最西格点所在的经度. 若在 90E 以西则统一记为 90E, 若在某月不存在 588 dagpm 等值线, 则以该月的 1951 - 2010 年历史最大值代替.

示例:
读入 hgt 文件以及 uwnd 文件, 指定时间 2014 年 10 月

In [ ]:
!./wpshi.py -hgt ~/Downloads/hgt.mon.mean.nc -uwnd ~/Downloads/uwnd.mon.mean.nc -y 2014 -m 10

输出:
(92, 173.0, 20.0, 100.0)

分别表示: 面积指数, 强度指数, 脊线指数, 西伸脊点